# API 확인 작업

In [2]:
import requests

key = "*****"
API_HOST = f"https://www.koreaexim.go.kr/site/program/financial/exchangeJSON?authkey={key}&data=AP01"
response = requests.get(API_HOST)

In [3]:
data = response.json()
len(data)

23

# 데이터 전처리

In [21]:
from datetime import datetime 
print(datetime.now())

2024-01-11 15:56:58.907171


In [117]:
import pandas as pd

trans_df = pd.DataFrame(data)[["deal_bas_r", "cur_nm"]]
trans_df.columns = ['exchange_rate', 'cur_nm']
trans_df['currency_unit'] = trans_df['cur_nm'].str.split().str[1]
trans_df['country'] = trans_df['cur_nm'].str.split().str[0]
del trans_df['cur_nm']

trans_df.head()

,exchange_rate,currency_unit,country
0,359.42,디르함,아랍에미리트
1,884.47,달러,호주
2,"3,501.68",디나르,바레인
3,991.29,달러,브루나이
4,986.7,달러,캐나다


In [12]:
trans_df['cur_nm'].str.split().str[1]

0     디르함
1      달러
2     디나르
3      달러
4      달러
5      프랑
6     NaN
7     크로네
8     NaN
9     파운드
10     달러
11    루피아
12      옌
13      원
14    디나르
15    링기트
16    크로네
17     달러
18     리얄
19    크로나
20     달러
21     바트
22     달러
Name: cur_nm, dtype: object

# DB와 비교

In [40]:
change_country = {'사우디': ['사우디아라비아'],  '위안화' : ['중국'], '덴마아크' : ['덴마아크'], '말레이지아' : ['말레이시아'],
                  '유로': ['네덜란드', '스페인', '독일', '아일랜드', '포르투갈', '이탈리아', '프랑스', '오스트리아', '룩셈부르크']}

In [118]:
delete_index = []
for exchange_rate, currency_unit, country in trans_df.values:
    if country in change_country.keys():
        country_list = change_country[country]
        for country_name in country_list:
            print(country, country_name)
            currency_unit = country if pd.isna(currency_unit) else currency_unit
            trans_df.loc[len(trans_df)] = [exchange_rate, currency_unit, country_name]
        
        delete_index.append(trans_df[trans_df['country']==country].index[0])

trans_df.drop(delete_index, axis=0, inplace=True)
trans_df

위안화 중국
덴마아크 덴마아크
유로 네덜란드
유로 스페인
유로 독일
유로 아일랜드
유로 포르투갈
유로 이탈리아
유로 프랑스
유로 오스트리아
유로 룩셈부르크
말레이지아 말레이시아
사우디 사우디아라비아


,exchange_rate,currency_unit,country
0,359.42,디르함,아랍에미리트
1,884.47,달러,호주
2,"3,501.68",디나르,바레인
3,991.29,달러,브루나이
4,986.7,달러,캐나다
5,"1,551.78",프랑,스위스
9,"1,682.07",파운드,영국
10,168.82,달러,홍콩
11,8.48,루피아,인도네시아
12,906.6,옌,일본


# 에러 확인

1. 2번째 task에서 return값으로 dataframe 주면 에러  
   -> numpy의 array도 안된다. **list로 치환**
2. 환율이 세자리 단위로 쉼표가 있음. INSERT 문에서 여러 개로 인식 함.  
   -> 쉼표 제거
3. TEMP table에 date 값이 default로 들어가지 않음  
   -> TEMP 만들 때 AS SELECT로 만든다고 해서 속성까지 받는다고 착각했다.  
   -> TEMP 테이블에 데이터를 insert할 때 현재 데이터를 넣어준다.
4. `ROW_NUMBER()` 사용 시 `ORDER BY`는 필수!
   -> date를 기준으로 ORDER BY DESC 꼭 사용할 것!

In [21]:
import requests
import pandas as pd

api_key = '****'
url = f"https://www.koreaexim.go.kr/site/program/financial/exchangeJSON?authkey={api_key}&data=AP01"
response = requests.get(url)
data = response.json()

# 필요한 정보 불러오기
trans_df = pd.DataFrame(data)[["deal_bas_r", "cur_nm"]]
trans_df.columns = ['exchange_rate', 'cur_nm']
trans_df['currency_unit'] = trans_df['cur_nm'].str.split().str[1]
trans_df['country'] = trans_df['cur_nm'].str.split().str[0]
trans_df['exchange_rate'] = trans_df['exchange_rate'].str.replace(',', '')   # 쉼표가 있어서 여러개의 문자로 인식됨
del trans_df['cur_nm']

# 변환 필요한 나라
change_country = {'사우디': ['사우디아라비아'],  '위안화' : ['중국'], '덴마아크' : ['덴마아크'], '말레이지아' : ['말레이시아'],
                    '유로': ['네덜란드', '스페인', '독일', '아일랜드', '포르투갈', '이탈리아', '프랑스', '오스트리아', '룩셈부르크']}

# 변환이 필요한 나라들 변환
delete_index = []
for exchange_rate, currency_unit, country  in trans_df.values:
    if country in change_country.keys():
        country_list = change_country[country]
        for country_name in country_list:
            currency_unit = country if pd.isna(currency_unit) else currency_unit
            trans_df.loc[len(trans_df)] = [exchange_rate, currency_unit, country_name]
        
        delete_index.append(trans_df[trans_df['country']==country].index[0])

# 변환 후 필요없는 행 삭제
trans_df.drop(delete_index, axis=0, inplace=True)

In [29]:
from datetime import datetime

now = datetime.now()
now.date()

datetime.date(2024, 1, 11)

In [32]:
df = trans_df.values.tolist()
for exchange_rate, currency_unit, country in df:
            sql = f"INSERT INTO t VALUES ('{now.date()}', '{country}', {exchange_rate}, '{currency_unit}', '{now}');"
            print(sql)

INSERT INTO t VALUES ('2024-01-11', '아랍에미리트', 359.42, '디르함', '2024-01-11 23:30:55.823330');
INSERT INTO t VALUES ('2024-01-11', '호주', 884.47, '달러', '2024-01-11 23:30:55.823330');
INSERT INTO t VALUES ('2024-01-11', '바레인', 3501.68, '디나르', '2024-01-11 23:30:55.823330');
INSERT INTO t VALUES ('2024-01-11', '브루나이', 991.29, '달러', '2024-01-11 23:30:55.823330');
INSERT INTO t VALUES ('2024-01-11', '캐나다', 986.7, '달러', '2024-01-11 23:30:55.823330');
INSERT INTO t VALUES ('2024-01-11', '스위스', 1551.78, '프랑', '2024-01-11 23:30:55.823330');
INSERT INTO t VALUES ('2024-01-11', '영국', 1682.07, '파운드', '2024-01-11 23:30:55.823330');
INSERT INTO t VALUES ('2024-01-11', '홍콩', 168.82, '달러', '2024-01-11 23:30:55.823330');
INSERT INTO t VALUES ('2024-01-11', '인도네시아', 8.48, '루피아', '2024-01-11 23:30:55.823330');
INSERT INTO t VALUES ('2024-01-11', '일본', 906.6, '옌', '2024-01-11 23:30:55.823330');
INSERT INTO t VALUES ('2024-01-11', '한국', 1, '원', '2024-01-11 23:30:55.823330');
INSERT INTO t VALUES ('2024-01-11',